In [ ]:
import numpy as np
import pandas as pd

# https://github.com/sublee/elo/blob/master/elo.py
"""
    elo
    ~~~
    The Elo rating system.
    :copyright: (c) 2012 by Heungsub Lee
    :license: BSD, see LICENSE for more details.
"""
from datetime import datetime
import inspect

__version__  = '0.1.dev'
__all__ = ['Elo', 'Rating', 'CountedRating', 'TimedRating', 'rate', 'adjust',
           'expect', 'rate_1vs1', 'adjust_1vs1', 'quality_1vs1', 'setup',
           'global_env', 'WIN', 'DRAW', 'LOSS', 'K_FACTOR', 'RATING_CLASS',
           'INITIAL', 'BETA']


#: The actual score for win.
WIN = 1.01
#: The actual score for draw.
DRAW = 0.5
#: The actual score for loss.
LOSS = 0.

#: Default K-factor.
K_FACTOR = 10
#: Default rating class.
RATING_CLASS = float
#: Default initial rating.
INITIAL = 1300
#: Default Beta value.
BETA = 170


class Rating(object):

    try:
        __metaclass__ = __import__('abc').ABCMeta
    except ImportError:
        # for Python 2.5
        pass

    value = None

    def __init__(self, value=None):
        if value is None:
            value = global_env().initial
        self.value = value

    def rated(self, value):
        """Creates a :class:`Rating` object for the recalculated rating.
        :param value: the recalculated rating value.
        """
        return type(self)(value)

    def __int__(self):
        """Type-casting to ``int``."""
        return int(self.value)

    def __long__(self):
        """Type-casting to ``long``."""
        return long(self.value)

    def __float__(self):
        """Type-casting to ``float``."""
        return float(self.value)

    def __nonzero__(self):
        """Type-casting to ``bool``."""
        return bool(int(self))

    def __eq__(self, other):
        return float(self) == float(other)

    def __lt__(self, other):
        """Is Rating < number.
        :param other: the operand
        :type other: number
        """
        return self.value < other

    def __le__(self, other):
        """Is Rating <= number.
        :param other: the operand
        :type other: number
        """
        return self.value <= other

    def __gt__(self, other):
        """Is Rating > number.
        :param other: the operand
        :type other: number
        """
        return self.value > other

    def __ge__(self, other):
        """Is Rating >= number.
        :param other: the operand
        :type other: number
        """
        return self.value >= other

    def __iadd__(self, other):
        """Rating += number.
        :param other: the operand
        :type other: number
        """
        self.value += other
        return self

    def __isub__(self, other):
        """Rating -= number.
        :param other: the operand
        :type other: number
        """
        self.value -= other
        return self

    def __repr__(self):
        c = type(self)
        ext_params = inspect.getargspec(c.__init__)[0][2:]
        kwargs = ', '.join('%s=%r' % (param, getattr(self, param))
                           for param in ext_params)
        if kwargs:
            kwargs = ', ' + kwargs
        args = ('.'.join([c.__module__, c.__name__]), self.value, kwargs)
        return '%s(%.3f%s)' % args


try:
    Rating.register(float)
except AttributeError:
    pass


class CountedRating(Rating):
    """Increases count each rating recalculation."""

    times = None

    def __init__(self, value=None, times=0):
        self.times = times
        super(CountedRating, self).__init__(value)

    def rated(self, value):
        rated = super(CountedRating, self).rated(value)
        rated.times = self.times + 1
        return rated


class TimedRating(Rating):
    """Writes the final rated time."""

    rated_at = None

    def __init__(self, value=None, rated_at=None):
        self.rated_at = rated_at
        super(TimedRating, self).__init__(value)

    def rated(self, value):
        rated = super(TimedRating, self).rated(value)
        rated.rated_at = datetime.utcnow()
        return rated


class Elo(object):

    def __init__(self, k_factor=K_FACTOR, rating_class=RATING_CLASS,
                 initial=INITIAL, beta=BETA):
        self.k_factor = k_factor
        self.rating_class = rating_class
        self.initial = initial
        self.beta = beta

    def expect(self, rating, other_rating):
        """The "E" function in Elo. It calculates the expected score of the
        first rating by the second rating.
        """
        # http://www.chess-mind.com/en/elo-system
        diff = float(other_rating) - float(rating)
        f_factor = 2 * self.beta  # rating disparity
        return 1. / (1 + 11 ** (diff / f_factor))

    def adjust(self, rating, series):
        """Calculates the adjustment value."""
        return sum(score - self.expect(rating, other_rating)
                   for score, other_rating in series)

    def rate(self, rating, series):
        """Calculates new ratings by the game result series."""
        rating = self.ensure_rating(rating)
        k = self.k_factor(rating) if callable(self.k_factor) else self.k_factor
        new_rating = float(rating) + k * self.adjust(rating, series)
        if hasattr(rating, 'rated'):
            new_rating = rating.rated(new_rating)
        return new_rating

    def adjust_1vs1(self, rating1, rating2, drawn=False):
        return self.adjust(rating1, [(DRAW if drawn else WIN, rating2)])

    def rate_1vs1(self, rating1, rating2, drawn=False):
        scores = (DRAW, DRAW) if drawn else (WIN, LOSS)
        return (self.rate(rating1, [(scores[0], rating2)]),
                self.rate(rating2, [(scores[1], rating1)]))

    def quality_1vs1(self, rating1, rating2):
        return 2 * (0.5 - abs(0.5 - self.expect(rating1, rating2)))

    def create_rating(self, value=None, *args, **kwargs):
        if value is None:
            value = self.initial
        return self.rating_class(value, *args, **kwargs)

    def ensure_rating(self, rating):
        if isinstance(rating, self.rating_class):
            return rating
        return self.rating_class(rating)

    def make_as_global(self):
        """Registers the environment as the global environment.
        >>> env = Elo(initial=2000)
        >>> Rating()
        elo.Rating(1200.000)
        >>> env.make_as_global()  #doctest: +ELLIPSIS
        elo.Elo(..., initial=2000.000, ...)
        >>> Rating()
        elo.Rating(2000.000)
        But if you need just one environment, use :func:`setup` instead.
        """
        return setup(env=self)

    def __repr__(self):
        c = type(self)
        rc = self.rating_class
        if callable(self.k_factor):
            f = self.k_factor
            k_factor = '.'.join([f.__module__, f.__name__])
        else:
            k_factor = '%.3f' % self.k_factor
        args = ('.'.join([c.__module__, c.__name__]), k_factor,
                '.'.join([rc.__module__, rc.__name__]), self.initial, self.beta)
        return ('%s(k_factor=%s, rating_class=%s, '
                'initial=%.3f, beta=%.3f)' % args)


def rate(rating, series):
    return global_env().rate(rating, series)


def adjust(rating, series):
    return global_env().adjust(rating, series)


def expect(rating, other_rating):
    return global_env().expect(rating, other_rating)


def rate_1vs1(rating1, rating2, drawn=False):
    return global_env().rate_1vs1(rating1, rating2, drawn)


def adjust_1vs1(rating1, rating2, drawn=False):
    return global_env().adjust_1vs1(rating1, rating2, drawn)


def quality_1vs1(rating1, rating2):
    return global_env().quality_1vs1(rating1, rating2)


def setup(k_factor=K_FACTOR, rating_class=RATING_CLASS,
          initial=INITIAL, beta=BETA, env=None):
    if env is None:
        env = Elo(k_factor, rating_class, initial, beta)
    global_env.__elo__ = env
    return env


def global_env():
    """Gets the global Elo environment."""
    try:
        global_env.__elo__
    except AttributeError:
        # setup the default environment
        setup()
    return global_env.__elo__
# -------------------------------------------------------

train = pd.read_csv('TourneyCompactResults.csv')

elo = Elo(125)

team = {}

for index, row in train.iterrows():
    t1 = row['Wteam']
    t2 = row['Lteam']
    if not t1 in team: team[t1] = 1000.0
    if not t2 in team: team[t2] = 1000.0

    (team[t1], team[t2]) = elo.rate_1vs1(team[t1], team[t2])
print(team)

elo = Elo(140)

team2 = {}

for index, row in train.iterrows():
    t1 = row['Wteam']
    t2 = row['Lteam']
    if not t1 in team2: team2[t1] = 1020.0
    if not t2 in team2: team2[t2] = 1020.0

    (team2[t1], team2[t2]) = elo.rate_1vs1(team2[t1], team2[t2])

preds = pd.read_csv('SampleSubmission.csv')
prediction = np.zeros((preds.shape[0], 1))
i = 0
for index, row in preds.iterrows():
    p = list(map(int, str.split(str(row['Id']), '_')))
    prediction[i] = 0.5 + 0.3*(team[p[1]] - team[p[2]]) / 480 + 0.7*(team2[p[1]] - team2[p[2]])/520
    i += 1

preds['Pred'] = np.clip(prediction, 0.05, 0.94)
preds.to_csv('Prediction.csv', index=False)


In [ ]:
def Ynikal(r,r1):
    r2 = list(r) + list(r1)
    r2.sort()
    i=0
    while i<(len(r2)-1):
        if r2[i] == r2[i+1]:
            del(r2[i])
            del(r2[i])
            i=0
        else:
            i = i+1
    for j in range(len(r2)):
        team[r2[j]] = 900
        team2[r2[j]] = 1000

In [2]:
Mas1 = pd.read_csv('TourneyCompactResults.csv')
Mas2 = pd.read_csv('RegularSeasonCompactResults.csv')
train = pd.concat([Mas1,Mas2])
for j in range(13):
    train1 = train[train['Season'] < 2016 - j].copy()
    elo = Elo(125)

    team = {}

    for index, row in train1.iterrows():
        t1 = row['Wteam']
        t2 = row['Lteam']
        if not t1 in team: team[t1] = 1000.0
        if not t2 in team: team[t2] = 1000.0

        (team[t1], team[t2]) = elo.rate_1vs1(team[t1], team[t2])
    elo = Elo(140)

    team2 = {}

    for index, row in train1.iterrows():
        t1 = row['Wteam']
        t2 = row['Lteam']
        if not t1 in team2: team2[t1] = 1020.0
        if not t2 in team2: team2[t2] = 1020.0

        (team2[t1], team2[t2]) = elo.rate_1vs1(team2[t1], team2[t2])

    
    df1 = pd.read_csv('RegularSeasonDetailedResults.csv')
    df1['REGTUR'] = 1
    df2 = pd.read_csv('TourneyDetailedResults.csv')
    df2['REGTUR'] = 0
    RegularSeasonDetailedResults = pd.concat([df1,df2])
    test = pd.DataFrame(data = [], columns=['Team1','Team2'])
    test['Team1'] = RegularSeasonDetailedResults[RegularSeasonDetailedResults['Season'] == 2016-j]['Wteam']
    test['Team2'] = RegularSeasonDetailedResults[RegularSeasonDetailedResults['Season'] == 2016-j]['Lteam']
    test['Season'] = 2016-j
    prediction = np.zeros(len(test))
    i = 0
    #убираем уникальные значения
    train = pd.read_csv('RegularSeasonCompactResults.csv')
    trainD = train[train['Season'] == 2016-j]
    trainD1 = train[train['Season'] == 2016-j-1]
    r = trainD['Wteam'].unique()
    r1 = trainD1['Wteam'].unique()
    Ynikal(r,r1)
    
    for index, row in test.iterrows():
        prediction[i] = 0.5 + 0.3*(team[row[0]] - team[row[1]]) / 480 + 0.7*(team2[row[0]] - team2[row[1]])/520
        i += 1
    test['Pred'] = np.clip(prediction, 0.05, 0.94)
    test.to_csv('Prediction'+str(2016-j)+'.csv', index=False)
    Razmer = []
    Razmer = test['Team1'].unique()
    Rutung = pd.DataFrame(data = Razmer, columns=['Team'])
    Rutung['Season'] = 2016-j
    Rutung['rutung1'] = 0
    Rutung['rutung2'] = 0
    for k in range(len(Razmer)):
        Rutung['rutung1'][k] = team[Razmer[k]]
        Rutung['rutung2'][k] = team2[Razmer[k]]
    Rutung.to_csv('PredictionRUT'+str(2016-j)+'.csv', index=False)
    print 0

NameError: name 'Ynikal' is not defined

In [ ]:
df = np.zeros((2278,1))
testT = pd.DataFrame(data = df, columns=['Team1'])
testT['Team2'] = 0
testT['Season'] = 2016
preds = pd.read_csv('SampleSubmission.csv')
i=0
for index, row in preds.iterrows():
        p = list(map(int, str.split(str(row['Id']), '_')))
        testT['Team1'][i] = p[1]
        testT['Team2'][i] = p[2]
        testT['Season'][i] = p[0]
        i = i + 1